## Import libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import pandas as pd
from sklearn.datasets import make_circles
from matplotlib.colors import Normalize




In [2]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix,\
        accuracy_score, classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,GradientBoostingClassifier,StackingClassifier,ExtraTreesClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.naive_bayes import GaussianNB

from time import time
from datetime import timedelta

import seaborn as sns
from matplotlib import pyplot as plt

sns.set()

## Load dataset

In [3]:
df=pd.read_csv('kickafterpreprocess.csv', sep=";")

FileNotFoundError: ignored

In [ ]:
df.columns

In [ ]:
df.drop(['age_cat','VehOdo_cat','VehBCost_cat','WarrantyCost_cat'], axis=1, inplace=True)

In [ ]:
df.columns

# Modelling

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
from sklearn.datasets import make_circles
from matplotlib.colors import Normalize


# Función para visualizar un conjunto de datos en 2D
def plot_data(X, y):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color,
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")
    
# Función para visualizar de la superficie de decisión de un clasificador
def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

def plot_labels(n_folds, n_classes, list_labels):
    ind = np.arange(n_folds)
    width = 0.15
    
    countings = []
    for labels in list_labels:
        labels = np.array(labels)
        countings.append([np.count_nonzero(labels == x) for x in range(n_classes)])
    
    class_bars = []
    for cls in range(n_classes):
        class_bars.append([l[cls] for l in countings])
    
    fig, ax = pl.subplots()
    i = 0
    for class_bar in class_bars:
        ax.bar(ind + width*i, class_bar, width, label='Clase '+str(i))
        i += 1
        
    ax.set_xticks(ind + 2*width / 3)
    ax.set_xticklabels(['Pliegue {}'.format(k) for k in range(n_folds)])
    pl.legend(loc="best")
    pl.title("Etiquetas")

In [ ]:
from sklearn.preprocessing import StandardScaler
escalar = StandardScaler()

# Get dummies 
Auction_dummies =pd.get_dummies(df['Auction'], prefix='Auction')
SubModel_dummies = pd.get_dummies(df['SubModel'], prefix='SubModel')
Color_dummies = pd.get_dummies(df['Color'], prefix='Color')
Transmission_dummies = pd.get_dummies(df['Transmission'], prefix='Transmission')
WheelType_dummies = pd.get_dummies(df['WheelType'], prefix='WheelType')
Nationality_dummies = pd.get_dummies(df['Nationality'], prefix='Nationality')
Size_dummies = pd.get_dummies(df['Size'], prefix='Size')
VNST_dummies = pd.get_dummies(df['VNST'], prefix='VNST')
IsOnlineSale_dummies = pd.get_dummies(df['IsOnlineSale'], prefix='IsOnlineSale')
season_dummies = pd.get_dummies(df['season'], prefix='season')


num_data = df[['VehicleAge','VehOdo','VehBCost','WarrantyCost']]
escalar.fit(num_data)
numdata2=escalar.transform(num_data)
dfnum=pd.DataFrame(numdata2)
dfnum.columns =['VehicleAge','VehOdo','VehBCost','WarrantyCost']

#VehicleAge = df['VehicleAge'].reshape(0,1)
#VehOdo = df['VehOdo'].reshape(0,1)
#VehBCost =escalar.transform df['VehBCost'].reshape(0,1)
#WarrantyCost = escalar.transform df['WarrantyCost'].reshape(0,1)

#VehicleAge = escalar.fit_transform (df['IsBadBuy','VehicleAge'])
#VehOdo = escalar.transform (df['VehOdo'])
#VehBCost =escalar.transform (df['VehBCost'])
#WarrantyCost = escalar.transform (df['WarrantyCost'])

In [ ]:
df.columns

In [ ]:
df = pd.concat([df['IsBadBuy'],
                dfnum,
                Auction_dummies, 
                SubModel_dummies, 
                Color_dummies,
                Transmission_dummies,
                WheelType_dummies, 
                Nationality_dummies, 
                Size_dummies, 
                VNST_dummies,
                IsOnlineSale_dummies,
                season_dummies], axis='columns')
df.columns

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
y=df['IsBadBuy']
X=df.drop(columns='IsBadBuy')
X_learn, X_test, y_learn, y_test = train_test_split(X, y, test_size=0.20, random_state=2022)
X_train, X_val, y_train, y_val = train_test_split(X_learn, y_learn, test_size=0.20, random_state=2022)

In [ ]:
from sklearn.svm import LinearSVC
linear_kick = LinearSVC(max_iter =2000)
linear_kick.fit(X_train, y_train)

In [ ]:
print("Training error: {}".format(1-linear_kick.score(X_train, y_train)))

In [ ]:
from sklearn.svm import SVC

poly_svm = SVC(kernel='poly', degree=2)
poly_svm.fit(X_train, y_train);

In [ ]:
print("Training error: {}".format(1-poly_svm.score(X_train, y_train)))
print("Test error: {}".format(1-poly_svm.score(X_test, y_test)))

In [ ]:
grid = [i for i in range(-10, 9,5)]
param_grid = {'C': [2**i for i in grid], 'gamma': [2**i for i in grid]}
#param_grid = {'C': [0.125,1,10,20], 'gamma': [0.125,1,2]}
print(param_grid['C'])

In [ ]:
print(param_grid['gamma'])

In [ ]:
#from sklearn.model_selection import RandomizedSearchCV
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
from pandas.core.common import random_state
clf = RandomizedSearchCV(SVC(kernel='linear'), param_distributions = param_grid,
                               cv = 2, n_iter = 5, n_jobs=-1, verbose=True)
#clf = GridSearchCV(SVC(kernel='linear'), param_grid=param_grid, verbose=2,n_jobs =  -1)
clf.fit(X_train, y_train)

In [ ]:
cv_results = pd.DataFrame(clf.cv_results_)
cv_results


In [ ]:
scores = clf.cv_results_['mean_test_score'].reshape(len(param_grid['C']),
                                                    len(param_grid['gamma']))

In [ ]:
cv_results = cv_results[['param_C', 'param_gamma', 'mean_test_score']]
cv_results.sort_values(by='mean_test_score', ascending=False).head()

In [ ]:
print(clf.best_params_)

In [ ]:
print(clf.best_score_)

## Metrics

In [ ]:
def confusion(true, pred):
    """
    Function for pretty printing confusion matrices
    """
    pred = pd.Series(pred)
    true = pd.Series(true)
    
    true.name = 'target'
    pred.name = 'predicted'
    cm = pd.crosstab(true.reset_index(drop=True), pred.reset_index(drop=True))
    cm = cm[cm.index]
    return cm

In [ ]:
def compute_metrics(y_true,y_pred):
    accuracy = accuracy_score(y_true,y_pred)
    f1_score_1 = f1_score(y_true,y_pred,average='binary',pos_label=1)
    f1_score_0 = f1_score(y_true,y_pred,average='binary',pos_label=0)
    f1_score_macro = f1_score(y_true,y_pred,average='macro')
    return [accuracy,f1_score_1,f1_score_0,f1_score_macro]
    
    

results = pd.DataFrame(columns=['Accuracy', 'F1-score (class 1)', 'F1-score (class 0)', 'F1-score (macro avg)'])

In [ ]:
#pd.Series(np.where(df1.IsBadBuy.values == 'yes', 1, 0),
#          df1.index)




## SVM


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
from sklearn.datasets import make_circles
from matplotlib.colors import Normalize


# Función para visualizar un conjunto de datos en 2D
def plot_data(X, y):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color,
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")
    
# Función para visualizar de la superficie de decisión de un clasificador
def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

def plot_labels(n_folds, n_classes, list_labels):
    ind = np.arange(n_folds)
    width = 0.15
    
    countings = []
    for labels in list_labels:
        labels = np.array(labels)
        countings.append([np.count_nonzero(labels == x) for x in range(n_classes)])
    
    class_bars = []
    for cls in range(n_classes):
        class_bars.append([l[cls] for l in countings])
    
    fig, ax = pl.subplots()
    i = 0
    for class_bar in class_bars:
        ax.bar(ind + width*i, class_bar, width, label='Clase '+str(i))
        i += 1
        
    ax.set_xticks(ind + 2*width / 3)
    ax.set_xticklabels(['Pliegue {}'.format(k) for k in range(n_folds)])
    pl.legend(loc="best")
    pl.title("Etiquetas")

In [ ]:
metrics=pd.DataFrame.from_dict(df, orient='index', columns=['MSE', 'MAE'])
metrics.sort_values(["MSE", "MAE"], ascending=(True, False))

In [ ]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

from sklearn.metrics import confusion_matrix, \
                  classification_report, accuracy_score,  precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split,  KFold, cross_validate, GridSearchCV

In [ ]:
logreg = LogisticRegressionCV(Cs=20, random_state=1, cv = 10, scoring = 'accuracy')

logreg.fit(X_train, y_train)

In [ ]:
avg_crossval_scores = logreg.scores_[1].mean(axis=0)
idx = np.argmax(avg_crossval_scores)
best_C = logreg.Cs_[idx]
print(best_C)

In [ ]:
results_df = pd.DataFrame(index=[], columns= ['Accuracy', 'F1 Macro', 'Precision Macro', 'Recall Macro'])

In [ ]:
logreg = LogisticRegression(C=best_C)
cross_val_results = pd.DataFrame(cross_validate(logreg, X_train, y_train, cv = 5, scoring = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'] ))

results_df.loc['Logistic Regression',:] = cross_val_results[['test_accuracy', 'test_f1_macro',
       'test_precision_macro', 'test_recall_macro']].mean().values

results_df.sort_values(by='Accuracy', ascending=False)